<a href="https://colab.research.google.com/github/sklearner26/signlanguage/blob/main/NLP1_text_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shubhammaindola/harry-potter-books

Dataset URL: https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books
License(s): CC0-1.0
 88% 2.00M/2.28M [00:00<00:00, 2.64MB/s]
100% 2.28M/2.28M [00:00<00:00, 2.57MB/s]


In [ ]:
!unzip harry-potter-books.zip

Archive:  harry-potter-books.zip
  inflating: 01 Harry Potter and the Sorcerers Stone.txt  
  inflating: 02 Harry Potter and the Chamber of Secrets.txt  
  inflating: 03 Harry Potter and the Prisoner of Azkaban.txt  
  inflating: 04 Harry Potter and the Goblet of Fire.txt  
  inflating: 05 Harry Potter and the Order of the Phoenix.txt  
  inflating: 06 Harry Potter and the Half-Blood Prince.txt  
  inflating: 07 Harry Potter and the Deathly Hallows.txt  


In [ ]:
!pip install textdistance

In [ ]:
import numpy as np
import pandas as pd
import textdistance
import re
from collections import Counter

In [ ]:
words1 =[]
with open('/content/01 Harry Potter and the Sorcerers Stone.txt',encoding='utf-8') as f:
  data = f.read()
  data = data.lower()
  word = re.findall('\w+',data)#collecting all words
  words1+=word

In [ ]:
len(words1)

80807

In [ ]:
words2 =[]
with open('/content/02 Harry Potter and the Chamber of Secrets.txt',encoding='utf-8') as f:
  data = f.read()
  data = data.lower()
  word = re.findall('\w+',data)#collecting all words
  words2+=word

In [ ]:
len(words2)

88670

In [ ]:
words=words1+words2

In [ ]:
#remove duplicacy
V=set(words)

## Counting frequency of each words


In [ ]:
word_freq_dict=Counter(words)
word_freq_dict

Counter({'m': 242,
         'r': 4,
         'and': 4135,
         'mrs': 142,
         'dursley': 60,
         'of': 2924,
         'number': 24,
         'four': 67,
         'privet': 23,
         'drive': 27,
         'were': 641,
         'proud': 9,
         'to': 3902,
         'say': 140,
         'that': 1414,
         'they': 1233,
         'perfectly': 11,
         'normal': 17,
         'thank': 17,
         'you': 2145,
         'very': 331,
         'much': 131,
         'the': 7763,
         'last': 172,
         'people': 176,
         'd': 504,
         'expect': 25,
         'be': 749,
         'involved': 6,
         'in': 1994,
         'anything': 135,
         'strange': 34,
         'or': 196,
         'mysterious': 8,
         'because': 154,
         'just': 364,
         'didn': 356,
         't': 1569,
         'hold': 32,
         'with': 903,
         'such': 40,
         'nonsense': 8,
         'mr': 215,
         'was': 2453,
         'director': 2,
     

In [ ]:
#Top 10 frequently repeated words
word_freq_dict.most_common(10)

[('the', 7763),
 ('and', 4135),
 ('to', 3902),
 ('a', 3502),
 ('he', 3298),
 ('harry', 3002),
 ('of', 2924),
 ('was', 2453),
 ('it', 2391),
 ('s', 2238)]

In [ ]:
#now calculate relative probability of each word
N=sum(word_freq_dict.values())
prob={}
for w in word_freq_dict:
  prob[w]=word_freq_dict[w]/N

In [ ]:
prob


{'m': 0.0014279223729473617,
 'r': 2.3602022693344818e-05,
 'and': 0.02439859095924521,
 'mrs': 0.0008378718056137411,
 'dursley': 0.0003540303404001723,
 'of': 0.017253078588835062,
 'number': 0.00014161213616006892,
 'four': 0.0003953338801135257,
 'privet': 0.0001357116304867327,
 'drive': 0.00015931365318007753,
 'were': 0.0037822241366085073,
 'proud': 5.310455106002584e-05,
 'to': 0.023023773137357872,
 'say': 0.0008260707942670687,
 'that': 0.008343315022097394,
 'they': 0.00727532349522354,
 'perfectly': 6.490556240669826e-05,
 'normal': 0.00010030859644671548,
 'thank': 0.00010030859644671548,
 'you': 0.01265658466930616,
 'very': 0.001953067377874284,
 'much': 0.0007729662432070429,
 'the': 0.04580562554210896,
 'last': 0.0010148869758138273,
 'people': 0.001038488998507172,
 'd': 0.002973854859361447,
 'expect': 0.00014751264183340512,
 'be': 0.004419478749328818,
 'involved': 3.540303404001723e-05,
 'in': 0.011765608312632393,
 'anything': 0.0007965682659003876,
 'strange':

In [ ]:
#Now find simillarity using jaccard simillarity


8739

In [ ]:
#First way to find similarity
def autocorrect(word):
  if word in prob:
    print('Word is already present,no need of correction',word)
  else:
    sim=[]
    for w in word_freq_dict:
      sim.append((1-textdistance.Jaccard(qval=2).distance(word,w)))
    df=pd.DataFrame.from_dict(prob,orient='index').reset_index()
    df=df.rename(columns={'index':'Word',0:'Prob'})
    df['Similarity']=sim
    output=df.sort_values(['Similarity','Prob'],ascending=False).head()

    return output

In [ ]:
#second way to find similarity
def autocorrect(word):
  output = None
  if word in prob:
    print('Word is already present,no need of correction',word)
  else:
    similarity=[1-(textdistance.Jaccard(qval=3)).distance(w,word) for w in word_freq_dict.keys()]
    df=pd.DataFrame.from_dict(prob,orient='index').reset_index()
    df=df.rename(columns={'index':'Word',0:'Prob'})
    df['Similarity']=similarity
    output=df.sort_values(['Similarity','Prob'],ascending=False).head()
    return output

In [ ]:
autocorrect('hel')

,Word,Prob,Similarity
275,help,0.000413,0.666667
736,held,0.000230,0.666667
1103,heel,0.000018,0.666667
52,he,0.019460,0.500000
2880,hello,0.000124,0.500000


In [ ]:
autocorrect('hel')

,Word,Prob,Similarity
275,help,0.000413,0.500000
736,held,0.000230,0.500000
2880,hello,0.000124,0.333333
1836,shelf,0.000047,0.333333
1763,whelk,0.000006,0.333333
